# Natural Language Processing with Kaggle Quora Classification Competition

## Load Data

We needed to store the data in AWS S3 so that we can use sagemaker to run our machine learning models.
Link: https://www.kaggle.com/c/quora-insincere-questions-classification/data

In [1]:
from dask.distributed import Client, progress
client = Client('192.168.0.13:8786')

In [2]:
import dask.dataframe as dd

In [93]:
import boto3
import pandas as pd
# from sagemaker import get_execution_role

# # files: embeddings.zip, sample_submission.csv.zip, test.csv.zip, train.csv.zip

# role = get_execution_role()
# bucket='quora-kaggle'
# data_key = 'train.csv.zip'
# data_location = 's3://{}/{}'.format(bucket, data_key)

df_train1 = pd.read_csv("train.csv.zip")
df_test1 = pd.read_csv("test.csv.zip")
df_sample1 = pd.read_csv("sample_submission.csv.zip")
import zipfile
with zipfile.ZipFile("train.csv.zip","r") as zip_ref:
    zip_ref.extractall()
df_train = dd.read_csv("train.csv")

with zipfile.ZipFile("test.csv.zip","r") as zip_ref:
    zip_ref.extractall()
df_test = dd.read_csv("test.csv")
# data = pd.read_csv('file1.csv', error_bad_lines=False)


In [10]:
# !pip install s3fs

In [11]:
# import s3fs
# fs = s3fs.S3FileSystem(anon=True)
# fs.ls('quora-kaggle')
# # ['my-file.txt']
# # with fs.open('my-bucket/my-file.txt', 'rb') as f:
# #     print(f.read())

In [97]:
print(df_train1.shape)
df_train.head()


(1306122, 3)


,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [92]:
print(df_test1.shape)
df_test.head()


(56370, 2)


,qid,question_text
0,00014894849d00ba98a9,My voice range is A2-C5. My chest voice goes u...
1,000156468431f09b3cae,How much does a tutor earn in Bangalore?
2,000227734433360e1aae,What are the best made pocket knives under $20...
3,0005e06fbe3045bd2a92,Why would they add a hypothetical scenario tha...
4,00068a0f7f41f50fc399,What is the dresscode for Techmahindra freshers?


In [96]:
df_sample1.head()

,qid,prediction
0,00014894849d00ba98a9,0
1,000156468431f09b3cae,0
2,000227734433360e1aae,0
3,0005e06fbe3045bd2a92,0
4,00068a0f7f41f50fc399,0


## Import NLP Libraries

In [14]:
# !pip install nltk

In [15]:
import sys
import nltk
import sklearn
import pandas
import numpy as np

## Preprocess Data

In [51]:
#separate data into our labels --> Y, and texts --> text_messages
Y = df_train['target']
#Y
# processed = dd.from_array(processed, chunksize=1000)
text_messages = dd.from_array(df_train['question_text'], chunksize = 1500)
# text_messages

In [52]:
# use regular expressions to replace email addresses, URLs, phone numbers, other numbers
#link: http://regexlib.com/?AspxAutoDetectCookieSupport=1

# Replace email addresses with 'email'
processed = text_messages.str.replace(r'^.+@[^\.].*\.[a-z]{2,}$',
                                 'emailaddress')

# Replace URLs with 'webaddress'
processed = processed.str.replace(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$',
                                  'webaddress')

# Replace money symbols with 'moneysymb' (£ can by typed with ALT key + 156)
processed = processed.str.replace(r'£|\$', 'moneysymb')
    
# Replace 10 digit phone numbers (formats include paranthesis, spaces, no spaces, dashes) with 'phonenumber'
processed = processed.str.replace(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$',
                                  'phonenumbr')
    
# Replace numbers with 'numbr'
processed = processed.str.replace(r'\d+(\.\d+)?', 'numbr')

# note: we end in 'r' instead of 'er' to prevent lemmatization 

In [53]:
# Remove punctuation
processed = processed.str.replace(r'[^\w\d\s]', ' ')

# Replace whitespace between terms with a single space
processed = processed.str.replace(r'\s+', ' ')

# Remove leading and trailing whitespace
processed = processed.str.replace(r'^\s+|\s+?$', '')

In [54]:
# change words to lower case - Hello, HELLO, hello are all the same word
processed = processed.str.lower()
print(processed)

Dask Series Structure:
npartitions=871
0          object
1500          ...
            ...  
1305000       ...
1306121       ...
dtype: object
Dask Name: str-lower, 8710 tasks


In [55]:
from nltk.corpus import stopwords
nltk.download('stopwords')

# remove stop words from text messages

stop_words = set(stopwords.words('english'))

processed = processed.apply(lambda x: ' '.join(
    term for term in x.split() if term not in stop_words))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/MacBookPro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [56]:
# Remove word stems using a Porter stemmer
ps = nltk.PorterStemmer()

processed = processed.apply(lambda x: ' '.join(
    ps.stem(term) for term in x.split()))

## Generating Features

In [25]:
help(dd.from_array)

Help on function from_array in module dask.dataframe.io.io:

from_array(x, chunksize=50000, columns=None)
    Read any slicable array into a Dask Dataframe
    
    Uses getitem syntax to pull slices out of the array.  The array need not be
    a NumPy array but must support slicing syntax
    
        x[50000:100000]
    
    and have 2 dimensions:
    
        x.ndim == 2
    
    or have a record dtype:
    
        x.dtype == [('name', 'O'), ('balance', 'i8')]



In [33]:
#use dask to process processed list to array

# processed = dd.from_array(processed, chunksize=1000)

In [64]:
help(processed.map_overlap)

Help on method map_overlap in module dask.dataframe.core:

map_overlap(func, before, after, *args, **kwargs) method of dask.dataframe.core.Series instance
    Apply a function to each partition, sharing rows with adjacent partitions.
    
    This can be useful for implementing windowing functions such as
    ``df.rolling(...).mean()`` or ``df.diff()``.
    
    Parameters
    ----------
    func : function
        Function applied to each partition.
    before : int
        The number of rows to prepend to partition ``i`` from the end of
        partition ``i - 1``.
    after : int
        The number of rows to append to partition ``i`` from the beginning
        of partition ``i + 1``.
    args, kwargs :
        Arguments and keywords to pass to the function. The partition will
        be the first argument, and these will be passed *after*.
    meta : pd.DataFrame, pd.Series, dict, iterable, tuple, optional
        An empty ``pd.DataFrame`` or ``pd.Series`` that matches the dtypes
 

In [81]:
type(processed)

dask.dataframe.core.Series

In [43]:
# processed = ['something message here', 'there are other messages are going to be another window']
# word_tokenize(['something in here'])

In [85]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')
# create bag-of-words
all_words = []
# print(type(processed))
# for message in processed:
#     words = word_tokenize(message)
#     for w in words:
#         all_words.append(w)

x=processed.map(lambda x: word_tokenize(x))
print(type(x))
y=x.map(lambda z: all_words.append(z))  
y.compute()
all_words = nltk.FreqDist(all_words)



[nltk_data] Downloading package punkt to
[nltk_data]     /Users/MacBookPro/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


<class 'dask.dataframe.core.Series'>


NotImplementedError: 

In [ ]:
# print the total number of words and the 15 most common words
print('Number of words: {}'.format(len(all_words)))
print('Most common words: {}'.format(all_words.most_common(15)))

In [ ]:
# use the 1500 most common words as features
word_features = list(all_words.keys())[:1500]

#THIS CAN BE CHANGED

In [ ]:
# The find_features function will determine which of the 1500 word features are contained in the review
def find_features(message):
    words = word_tokenize(message)
    features = {}
    for word in word_features:
        features[word] = (word in words)

    return features

# Lets see an example!
features = find_features(processed[0])
for key, value in features.items():
    if value == True:
        print(key)

In [ ]:
# Now lets do it for all the messages
messages = zip(processed, Y)

# define a seed for reproducibility
# seed = 1
# np.random.seed = seed
# np.random.shuffle(messages)

# call find_features function for each SMS message
featuresets = [(find_features(text), label) for (text, label) in messages]

In [ ]:
# we can split the featuresets into training and testing datasets using sklearn
from sklearn import model_selection

# split the data into training and testing datasets
training, testing = model_selection.train_test_split(featuresets, test_size = 0.25, random_state=seed)

In [ ]:
print(len(training))
print(len(testing))

## Scikit-Learn Classifiers with NLTK

In [ ]:
# We can use sklearn algorithms in NLTK
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.svm import SVC

model = SklearnClassifier(SVC(kernel = 'linear'))

# train the model on the training data
model.train(training)

# and test on the testing dataset!
accuracy = nltk.classify.accuracy(model, testing)*100
print("SVC Accuracy: {}".format(accuracy))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Define models to train
names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LogisticRegression(),
    SGDClassifier(max_iter = 100),
    MultinomialNB(),
    SVC(kernel = 'linear')
]

models = zip(names, classifiers)

for name, model in models:
    nltk_model = SklearnClassifier(model)
    nltk_model.train(training)
    accuracy = nltk.classify.accuracy(nltk_model, testing)*100
    print("{} Accuracy: {}".format(name, accuracy))

In [ ]:
# Ensemble methods - Voting classifier
from sklearn.ensemble import VotingClassifier

names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LogisticRegression(),
    SGDClassifier(max_iter = 100),
    MultinomialNB(),
    SVC(kernel = 'linear')
]

models = zip(names, classifiers)

nltk_ensemble = SklearnClassifier(VotingClassifier(estimators = models, voting = 'hard', n_jobs = -1))
nltk_ensemble.train(training)
accuracy = nltk.classify.accuracy(nltk_model, testing)*100
print("Voting Classifier: Accuracy: {}".format(accuracy))

In [ ]:
# make class label prediction for testing set
txt_features, labels = zip(*testing)

prediction = nltk_ensemble.classify_many(txt_features)

In [ ]:
# print a confusion matrix and a classification report
print(classification_report(labels, prediction))

pd.DataFrame(
    confusion_matrix(labels, prediction),
    index = [['actual', 'actual'], ['ham', 'spam']],
    columns = [['predicted', 'predicted'], ['ham', 'spam']])